In [5]:
%pip install h2o sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19559406/tmp0bhqn488
  JVM stdout: /state/partition1/job-19559406/tmp0bhqn488/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19559406/tmp0bhqn488/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_mk7516_s5k21n
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.543 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup(531)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(379, 14)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_4_AutoML_1_20220514_20916


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,22.0,4444.0,1275723.0,1.0,10.0,7.836859,1.0,28.0,18.078758




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.7023942570163303
RMSE: 0.838089647362578
LogLoss: 23.915626634259574
Mean Per-Class Error: 0.6968973270953468
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,3,4,5,6,7,8,9,10,...,216,219,221,222,223,225,227,228,Error,Rate
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1 / 1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,2 / 2
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,2 / 2
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1 / 1
4,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,1 / 3
5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,1 / 4
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,2 / 2
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,1 / 2
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,2 / 2
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,1 / 2



See the whole table with table.as_data_frame()

Top-10 Hit Ratios: 


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.026351,0.013114,0.026316,0.013158,0.039474,0.039474,0.013333
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.973649,0.013114,0.973684,0.986842,0.960526,0.960526,0.986667
3,err_count,73.800000,0.836660,74.000000,75.000000,73.000000,73.000000,74.000000
4,logloss,22.240265,14.139822,33.188606,6.639658,6.889792,31.608646,32.874620
5,max_per_class_error,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,mean_per_class_accuracy,0.684851,0.011991,0.685644,0.674257,0.671617,0.699670,0.693069
7,mean_per_class_error,0.315148,0.011991,0.314356,0.325743,0.328383,0.300330,0.306931
8,mse,0.968461,0.008196,0.965806,0.972982,0.972092,0.955395,0.976031
9,pr_auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-05-14 02:19:51,30.659 sec,0.0,0.995050,5.308268,0.994723,NaN,NaN
1,,2022-05-14 02:19:53,32.836 sec,5.0,0.798407,1.854797,0.419525,NaN,NaN
2,,2022-05-14 02:19:55,34.649 sec,10.0,0.721701,6.979018,0.372032,NaN,NaN
3,,2022-05-14 02:19:57,36.882 sec,15.0,0.848879,20.006251,0.722955,NaN,NaN
4,,2022-05-14 02:19:59,38.771 sec,20.0,0.838608,23.429054,0.704485,NaN,NaN
5,,2022-05-14 02:19:59,39.370 sec,22.0,0.838090,23.915627,0.699208,NaN,NaN



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,LSTAT,201.790375,1.000000,0.150419
1,DIS,187.880173,0.931066,0.140050
2,RM,183.948608,0.911583,0.137119
3,CRIM,173.412643,0.859370,0.129265
4,B,133.622147,0.662183,0.099605
5,AGE,113.292778,0.561438,0.084451
6,NOX,83.443840,0.413517,0.062201
7,TAX,61.076160,0.302671,0.045527
8,ZN,56.106232,0.278042,0.041823
9,INDUS,43.412746,0.215138,0.032361


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
GBM_4_AutoML_1_20220514_20916,0.989052,17.4434,0.983665,0.967598
StackedEnsemble_BestOfFamily_1_AutoML_1_20220514_20916,0.993228,11.0919,0.993801,0.98764
StackedEnsemble_AllModels_1_AutoML_1_20220514_20916,0.994186,6.27194,0.992038,0.984139
GBM_5_AutoML_1_20220514_20916,0.994415,17.4907,0.989459,0.979029
StackedEnsemble_BestOfFamily_2_AutoML_1_20220514_20916,0.994796,10.7853,0.993868,0.987774
GLM_1_AutoML_1_20220514_20916,0.99505,9.58785,0.994991,0.990007
StackedEnsemble_AllModels_2_AutoML_1_20220514_20916,0.995081,5.59351,0.992173,0.984407
StackedEnsemble_BestOfFamily_3_AutoML_1_20220514_20916,0.995392,9.44658,0.994912,0.989851
GBM_1_AutoML_1_20220514_20916,0.995675,5.51277,0.99243,0.984917
XRT_1_AutoML_1_20220514_20916,0.996065,23.4084,0.98922,0.978556
